In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# import dash

In [2]:
df = pd.read_csv('rsc/videogame_sales.csv')
df = df.dropna(subset=['Year'])
df['Year'] = df['Year'].astype(int)
#Sales columns are in Millions

In [17]:
pubs = df['Publisher'].value_counts()
big_pubs = pubs.loc[pubs>100].index
recent_publisher_mkt_share = df.loc[df['Year'] >= 2010].groupby(['Publisher'])['Global_Sales'].sum().to_frame().reset_index()
recent_publisher_mkt_share = recent_publisher_mkt_share.sort_values('Global_Sales', ascending=False).reset_index(drop=True)
others_sales = recent_publisher_mkt_share.iloc[10:]['Global_Sales'].sum()
recent_publisher_mkt_share = recent_publisher_mkt_share.iloc[0:10]
recent_publisher_mkt_share = recent_publisher_mkt_share.append(
    {'Publisher': 'Others', 'Global_Sales': others_sales}, ignore_index=True)
df['year_bin'] = (df['Year'] //5) * 5 # left inclusive right exclusive
df['year_bin'] += 5 # shift to right edge
popular_genre_past_5_year = df.groupby(['year_bin','Genre'])['Global_Sales'].sum().reset_index('Genre')

In [4]:
column_styles = {'box-sizing':'border-box', 'width': '50%', 'display':'flex','flex-direction': 'column', 'flex':'1',
              'float': 'left', 'padding': '20px',
              'background-color':'inherit'}
figure_style = {'border-radius': '20px', 'background-color':'white', 'box-sizing':'inherit', 'padding': '20px', 'margin': '10px',
                 'display':'flex','flex-direction': 'column', 'flex':'1'}

In [41]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    # Left column Div
    html.Div([
        html.Div([
            html.H2("Platforms", style = {"font-family":"verdana"}),
            dash_table.DataTable(
                id='table_1',
                page_action='native',
                columns=[{"name": i, "id": i} for i in df.columns],
                page_size=10,
                style_table={'box-sizing': 'inherit', 'overflowX': 'auto', 'overflowY': 'auto'}),
            html.Label([
                "Select a Platform",
                dcc.Dropdown(
                    id='platform', clearable=False,
                    value='PC', options=[
                        {'label': p, 'value': p} for p in df['Platform'].unique()
                    ])
            ])
        ], style=figure_style),

        html.Div([
            html.H2("Genre", style = {"font-family":"verdana"}),
            dcc.Graph(id='figure_3'),
            html.Label([
                "Select a year",
                dcc.Slider(
                    id='year', 
                    min=popular_genre_past_5_year.index.min(),
                    max=popular_genre_past_5_year.index.max(),
                    step=None,
                    marks={y : str(y) for y in popular_genre_past_5_year.index},
                    value=popular_genre_past_5_year.index.min())
            ])
        ], style=figure_style)
    ],
        style=column_styles, className='left column'),

    # Right column Div
    html.Div([
        html.Div([
            html.H1("Competitor performance", style = {"font-family":"verdana"}),
            dcc.Graph(id='figure_2'),
            html.Label([
                "Select a Publisher",
                dcc.Dropdown(
                    id='publisher', clearable=False,
                    value='Nintendo', options=[
                        {'label': c, 'value': c}
                        for c in big_pubs  # Dropdown에 항목이 너무 많으면 안됨. -- 아예 안만들어줌.
                    ])
            ])
        ], style=figure_style),
        html.Div([
            html.H1("Current Market Share", style = {"font-family":"verdana"}),
            dcc.Graph(id='figure_4',
                      figure=px.pie(recent_publisher_mkt_share,
                                    values='Global_Sales',
                                    names='Publisher',
                                    title='Publisher Market share Since 2020',
                                    hole=0.3)),
        ], style=figure_style)
    ], style=column_styles, className='right column')

], style={'box-sizing': 'border-box', 'background-color': '#ECECEC', 'height': '100%', 'display': 'flex'},
    className='MainDiv')
# 'height': '100%', 'display':'flex' 이게 중요하네.


############ Figure 1 ############
target = 1


@app.callback(
    Output(f'table_{target}', 'data'),  # id, what to change
    [Input(f"platform", "value")]  # id, what to read
)
def update_table_1(platform):
    data = df.loc[df['Platform'] == platform]
    columns = [{"name": i, "id": i} for i in data.columns]
    return data.to_dict('records')


############ Figure 2 ############
@app.callback(
    Output(f'figure_2', 'figure'),  # id, what to change
    [Input("publisher", "value")]  # id, what to read
)
def update_figure_2(publisher):
    pulisher_sales_df = df.groupby(['Publisher', 'Year'])[['Global_Sales']].sum().reset_index('Year')
    return px.bar(
        pulisher_sales_df.loc[publisher], x="Year", y="Global_Sales",
        title="Publisher Sales"
    )


############ Figure 3 ############
@app.callback(
    Output(f'figure_3', 'figure'),  # id, what to change
    [Input(f"year", "value")]  # id, what to read
)
def update_figure_3(year):

    return px.bar(
        popular_genre_past_5_year.loc[[year]], x="Genre", y="Global_Sales",
        title="Populare Genre in the last 5 years"
    )


# Run app and display result inline in the notebook
app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/


In [25]:
popular_genre_past_5_year.loc[[2025]]

,Genre,Global_Sales
year_bin,,
2025,Simulation,0.29


In [117]:
{y : y for y in df['Year'].unique()}

{2006: 2006,
 1985: 1985,
 2008: 2008,
 2009: 2009,
 1996: 1996,
 1989: 1989,
 1984: 1984,
 2005: 2005,
 1999: 1999,
 2007: 2007,
 2010: 2010,
 2013: 2013,
 2004: 2004,
 1990: 1990,
 1988: 1988,
 2002: 2002,
 2001: 2001,
 2011: 2011,
 1998: 1998,
 2015: 2015,
 2012: 2012,
 2014: 2014,
 1992: 1992,
 1997: 1997,
 1993: 1993,
 1994: 1994,
 1982: 1982,
 2003: 2003,
 1986: 1986,
 2000: 2000,
 1995: 1995,
 2016: 2016,
 1991: 1991,
 1981: 1981,
 1987: 1987,
 1980: 1980,
 1983: 1983,
 2020: 2020,
 2017: 2017}